In [ ]:
import logging
import sys

from astropy.time import Time, TimeDelta
import numpy as np

import lsst.daf.butler as dafButler
from lsst_efd_client import EfdClient
from check_utils import check, check_float, check_near

run_logging = False

In [ ]:
if run_logging:
    stream_handler = logging.StreamHandler(sys.stdout)

    logger = logging.getLogger()
    logger.addHandler(stream_handler)
    logger.level = logging.NOTSET
    logging.getLogger("matplotlib").setLevel(logging.WARNING)

In [ ]:
# NTS
#dataPath = "/repo/LSSTComCam"
#efd_name = "ncsa_teststand_efd"
# RSP
dataPath = "/repo/main"
efd_name = "ldf_stable_efd"

collections = ["LSSTComCam/raw/all"]
instrument = "LSSTComCam"
butler = dafButler.Butler(dataPath, collections=collections, instrument=instrument)
client = EfdClient(efd_name)

In [ ]:
day_obs = 20210602
seq_num = 60
raft_name = "R22"
detector = 0
dataId = {"instrument": instrument, "detector.raft": raft_name, "detector.id": detector,
          "exposure.day_obs": day_obs, "exposure.seq_num": seq_num}
raw = butler.get('raw.metadata', dataId)

In [ ]:
header = raw.toDict()
#print(list(header.keys()))
#print(header)

In [ ]:
delta = TimeDelta(20.0, format="sec", scale="tai")
obs_beg = Time(header["DATE-BEG"], format="fits", scale="tai")
obs_end = Time(header["DATE-END"], format="fits", scale="tai")
exp_time = TimeDelta(float(header["EXPTIME"]), format="sec", scale="tai")
#print(obs_beg)
#print(header["DATE-BEG"])
obs_id = header["OBSID"]
#print(f"IMGTYPE: {header['IMGTYPE']}")
#print(f"EXPTIME: {header['EXPTIME']}")
#print(f"DARKTIME: {header['DARKTIME']}")
print(f"DATE-BEG: {header['DATE-BEG']}")
print(f"DATE-END: {header['DATE-END']}")

In [ ]:
start_int_df = await client.select_time_series("lsst.sal.CCCamera.logevent_startIntegration", "*",
                                               obs_beg, delta, is_window=True)
end_image_tel_df = await client.select_time_series("lsst.sal.CCCamera.logevent_endOfImageTelemetry", "*",
                                                   obs_end, delta, is_window=True)

In [ ]:
print(f"Start Integration Num Rows: {len(start_int_df)}")
print(f"End of Image Telemetry Num Rows: {len(end_image_tel_df)}")

In [ ]:
start_int = start_int_df.loc[start_int_df["imageName"] == obs_id]
end_image_tel = end_image_tel_df.loc[end_image_tel_df["imageName"] == obs_id]

In [ ]:
# Scales are UTC because incoming timestamps are already TAI and astropy screws them up if the scale is TAI.
date_obs = Time(end_image_tel["timestampDateObs"][0], format="unix", scale="utc")
date_end = Time(end_image_tel["timestampDateEnd"][0], format="unix", scale="utc")
date_start_int = Time(start_int["private_sndStamp"][0], format="unix", scale="utc")
date_eoit = Time(end_image_tel["private_sndStamp"][0], format="unix", scale="utc")
#print(date_start_int.isot)
#print(date_eoit.isot)

In [ ]:
check(header, date_obs.isot, "DATE-BEG")
check(header, date_end.isot, "DATE-END")

In [ ]:
date_diff = date_end - date_obs
if date_diff.sec >= float(header["EXPTIME"]):
    print("DATE-END - DATE-BEG OK")
else:
    print(f"Problem with DATE-END - DATE-BEG: {date_diff.sec} seconds")
    print(f"Exposure time: {header['EXPTIME']} seconds")

In [ ]:
shutter_motion_profile_df = await client.select_top_n("lsst.sal.CCCamera.logevent_endOfImageTelemetry", "*", 1,
                                                      time_cut=date_eoit.isot)

In [ ]:
check_float(header, start_int["exposureTime"][0], "EXPTIME")
check_float(header, end_image_tel["darkTime"][0], "DARKTIME")
check_float(header, shutter_motion_profile_df["measuredShutterOpenTime"][0], "SHUTTIME")

In [ ]:
current_target_df = await client.select_top_n("lsst.sal.MTPtg.logevent_currentTarget", "*", 1,
                                              time_cut=date_start_int.isot)
mount_positions_start_df = await client.select_top_n("lsst.sal.MTPtg.mount_position", "*", 1,
                                                     time_cut=date_start_int.isot)
mount_positions_end_df = await client.select_top_n("lsst.sal.MTPtg.mount_position", "*", 1,
                                                   time_cut=date_eoit.isot)

In [ ]:
if header["IMGTYPE"] in ["OBJECT", "ENGTEST"]:
    check(header, current_target_df["targetName"][0], "OBJECT")
    check_float(header, np.degrees(current_target_df["ra"])[0], "RA")
    check_float(header, np.degrees(current_target_df["declination"])[0], "DEC")
    check_float(header, current_target_df["rotPA"][0], "ROTPA")
    check_float(header, mount_positions_start_df["ra0"][0], "RASTART")
    check_float(header, mount_positions_end_df["ra0"][0], "RAEND")
    check_near(header, "RA", "RASTART")
    check_near(header, "RA", "RAEND")
    check_float(header, mount_positions_start_df["declination0"][0], "DECSTART")
    check_float(header, mount_positions_end_df["declination0"][0], "DECEND")
    check_near(header, "DEC", "DECSTART")
    check_near(header, "DEC", "DECEND")

In [ ]:
filter_pos_df = await client.select_top_n("lsst.sal.CCCamera.logevent_endSetFilter", "*", 1,
                                          time_cut=date_start_int.isot)

In [ ]:
check(header, filter_pos_df["filterType"][0], "FILTBAND")
check(header, filter_pos_df["filterName"][0], "FILTER")
check(header, filter_pos_df["filterSlot"][0], "FILTSLOT")
check(header, filter_pos_df["filterPosition"][0], "FILTPOS")

In [ ]:
focal_plane_info_df = await client.select_top_n("lsst.sal.CCCamera.logevent_focalPlaneSummaryInfo", "*", 1,
                                                time_cut=date_start_int.isot)
vacuum_df = await client.select_top_n("lsst.sal.CCCamera.vacuum", "*", 1,
                                      time_cut=date_start_int.isot)

In [ ]:
check(header, focal_plane_info_df["ccdManufacturer"][0], "CCD_MANU", index=detector)
check(header, focal_plane_info_df[f"ccdType{detector}"][0], "CCD_TYPE")
check(header, focal_plane_info_df["ccdManSerNum"][0], "CCD_SERN", index=detector)
check(header, focal_plane_info_df["ccdLSSTName"][0], "LSST_NUM", index=detector)
check(header, focal_plane_info_df["ccdSlot"][0], "CCDSLOT", index=detector)
check(header, focal_plane_info_df["raftBay"][0], "RAFTBAY", index=detector)
check(header, focal_plane_info_df["sequencerChecksum"][0], "SEQCKSUM", index=detector)
check(header, focal_plane_info_df["sequencerKey"][0], "SEQNAME", index=detector)
check(header, focal_plane_info_df["rebLSSTName"][0], "REBNAME", index=detector)
check(header, focal_plane_info_df["rebSerialNumber"][0], "CONTNUM", index=detector)
#check_float(header, fp_ccd_df["temp"][0], "CCDTEMP")
check_float(header, focal_plane_info_df[f"ccdTempSetPoint{detector}"][0], "TEMP_SET")